## Spam Classification


In [1]:
# Standard imports
import numpy as np
import pandas as pd

import spacy

# Sklearn
from sklearn.model_selection import train_test_split


# Built-in library
import re
import warnings

warnings.filterwarnings("ignore")

# pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Black code formatter (Optional)
%load_ext lab_black
# auto reload imports
%load_ext autoreload
%autoreload 2

In [7]:
def load_data(
    *, filename: str, sep: str = ",", encoding: str = "utf-8"
) -> pd.DataFrame:
    """This is used to load the data.
    Params;
        filename (str): The filepath.
        sep (str, default=","): The separator. e.g ",", "\t", etc
        encoding (str, default="utf-8): The encoding type.

    Returns:
        data (pd.DataFrame): The loaded dataframe.
    """
    data = pd.read_csv(filename, sep=sep, encoding=encoding)
    return data

In [3]:
# Download the data
!wget https://lazyprogrammer.me/course_files/spam.csv

--2023-03-19 14:38:07--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  1.76MB/s    in 0.3s    

2023-03-19 14:38:11 (1.76 MB/s) - ‘spam.csv’ saved [503663/503663]



In [8]:
# Load Data
fp = "spam.csv"
data = load_data(filename=fp, encoding="ISO-8859-1")
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives around here though",NaN,NaN,NaN
